In [83]:
import pymongo
from bs4 import BeautifulSoup
import time
import numpy as np
import pandas as pd
from  datetime import datetime, timedelta


In [181]:


def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

@timeit
def parse_auction_page(html):
    auction_title=html.split(qauction_id + " - ")[1]
    card_value, auction_title = auction_title.split(" ", 1)
    has_bids = "Bids" in auction_title
    has_gift_card = "Gift Card" in auction_title
    cash_value = auction_title.split("(")[1]
    cash_value = cash_value.split(")")[0][1:]
    if has_bids and has_gift_card:
        card_type, bid_value = auction_title.split (" Gift Card AND", 1)
        bid_value = bid_value.split(" Bids")[0][1:]
    elif has_gift_card and not has_bids:
        card_type = auction_title.split (" Gift Card")[0]
        bid_value = "0"  
    elif has_bids and not has_gift_card:
        card_type = "None"
        bid_value = card_value
        card_value = "$0"
    else:
        print("what is this?")               
    limited_allowed = "is NOT LIMITED" not in html
    auction_dict["cashvalue"] = int(cash_value)
    auction_dict["cardvalue"]=int(card_value[1:])
    auction_dict["bidvalue"]=int(bid_value)
    auction_dict["cardtype"]=card_type
    auction_dict["limited_allowed"]=limited_allowed

    
    
@timeit
def parse_auction_table_page(html):
    soup = BeautifulSoup(html, 'html.parser')
    tds = soup.find_all("td")
    tracking = str(tds[0]).split('%')[0].split()[1]
    last_update = str(tds[1]).split ("update: ")[1].split("</td>")[0]    
    run_time = str(tds[3]).split("runtime : ")[1].split("</td>")[0]
    auction_dict["tracking"] = float(tracking)
    auction_dict["auctiontime"] = datetime.strptime(last_update, '%m/%d/%Y %I:%M:%S %p') #last_update
    t = datetime.strptime(run_time, "%H:%M:%S")
    delta = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
    auction_dict["runtime"] = t.hour*3600+t.minute*60+t.second
    lock_price = html.split('<div id="div_lock_prcnt" style="display: none;">')[1].split("<")[0]  
    if (lock_price):
        lock_price = int(float(lock_price)*100)
    auction_dict["lock_price"] = lock_price
    
@timeit
def parse_history_page(html, bids_list):
    soup = BeautifulSoup(html, 'html.parser')
    tds = soup.find_all("td")[3:]
    for i in range(0, len(tds), 4):
        bid = tds[i].text.strip() 
        user = tds[i+2].text.strip() 
        bid_type = tds[i+3].text.strip()   
        is_bidomatic = bid_type=="Bidomatic"
        bids_list.append({"auction_id": int(qauction_id), "bid": int(bid), "user": user, "is_bidomatic": is_bidomatic})




In [182]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["penny"]
pages_collection = db["pages"]
auction_collection = db["auctions"]
bids_collection = db["bids"]

auction_list = []
    

#conn = pg2.connect(user='postgres',  dbname='penny', host='localhost', port='5432', password='')

#with conn.cursor() as cursor:
#    sql = "SELECT qauctionid from Auctions"
#    cursor.execute(sql)
#    already_got_qids = [qid[0] for qid in cursor.fetchall()]

#    print(already_got_qids)    

#    i = 0
for p in pages_collection.find({}):
    qauction_id = p['_id']
    if auction_collection.count_documents({"_id":int(qauction_id)})==0:
#    if int(qauction_id) not in already_got_qids:
        print(qauction_id, p['AuctionGroup'])
        auction_dict = {"_id": int(qauction_id)}
        bids_list = []
        parse_auction_page(p['Auction'])
        parse_auction_table_page(p['AuctionTable'])
        parse_history_page(p['AuctionHistory'], bids_list)
        auction_collection.insert_one(auction_dict,{"ordered":"True"})
        bids_collection.insert_many(bids_list)
#        myquery = { "_id": qauction_id }
#        newvalues = { "$set": auction_dict }
#        print(auction_dict)
#        break;
#        pages_collection.update_one(myquery, newvalues)
#        myquery = { "_id": qauction_id }
#        unsetlist = {k:1 for k in auction_dict.keys() if k not in ["qauctionid"]} 
#        unsetvalues = {"$unset":unsetlist}
#        print(unsetvalues)
#        pages_collection.update_one(myquery, unsetvalues)
        #auction_list.append(auction_dict)



#    for d in auction_list:
#        if d["tracking"] == 100.00:
#            sql = "INSERT INTO Auctions (qauctionid, cardvalue, bidvalue, tracking, auctiontime, runtime, limited_allowed, cashvalue) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
#            cursor.execute(sql, (d['qauctionid'], d['cardvalue'], d['bidvalue'], d['tracking'], d['auctiontime'], d['runtime'], d['limited_allowed'], d['cashvalue']))
#        print (d.auction_dict.count())
#        for k, v  in d.auction_dict.items():
#            print(k, v)
            
            
#    conn.commit()
 
    
    # connection is not autocommit by default. So you must commit to save
    # your changes.
     

265936045 amazon50bids20
'parse_auction_page'  0.06 ms
'parse_auction_table_page'  7.35 ms
'parse_history_page'  76.82 ms
585652307 amazon50bids20
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  13.30 ms
'parse_history_page'  264.23 ms
778393197 amazon50bids20
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  6.63 ms
'parse_history_page'  92.78 ms
643682635 amazon50bids20
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  6.46 ms
'parse_history_page'  123.71 ms
782287760 amazon50bids20
'parse_auction_page'  0.07 ms
'parse_auction_table_page'  4.39 ms
'parse_history_page'  6.51 ms
261136026 amazon50bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  5.94 ms
'parse_history_page'  30.86 ms
218837987 amazon50bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  12.84 ms
'parse_history_page'  703.23 ms
691662580 amazon50bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  10.83 ms
'parse_history_page'  203.74 ms
200966294 amazon50

'parse_history_page'  61.78 ms
444641853 amazon150bids30
'parse_auction_page'  0.06 ms
'parse_auction_table_page'  7.93 ms
'parse_history_page'  20.88 ms
651463316 amazon100bids30
'parse_auction_page'  0.07 ms
'parse_auction_table_page'  26.99 ms
'parse_history_page'  255.25 ms
774195265 amazon100bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  10.83 ms
'parse_history_page'  141.47 ms
502921069 amazon100bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  20.86 ms
'parse_history_page'  430.09 ms
917461267 amazon100bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  16.26 ms
'parse_history_page'  701.02 ms
917854562 amazon100bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  6.94 ms
'parse_history_page'  18.20 ms
633387725 amazon75bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  14.91 ms
'parse_history_page'  296.14 ms
233546875 amazon75bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  16.88 ms
'parse_

'parse_history_page'  79.85 ms
888942259 walmart50bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  26.35 ms
'parse_history_page'  265.98 ms
755723738 walmart50bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  3.78 ms
'parse_history_page'  7.61 ms
514984090 walmart50bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  8.12 ms
'parse_history_page'  74.89 ms
204808977 walmart50bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  11.67 ms
'parse_history_page'  99.49 ms
823079507 walmart50bids35
'parse_auction_page'  0.06 ms
'parse_auction_table_page'  17.27 ms
'parse_history_page'  781.68 ms
525474438 walmart50bids35
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  9.07 ms
'parse_history_page'  51.12 ms
166718084 walmart50bids35
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  8.09 ms
'parse_history_page'  51.10 ms
406360164 walmart50bids35
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  7.15 ms
'parse_histo

'parse_history_page'  124.99 ms
549570810 amazon25bids20
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  3.03 ms
'parse_history_page'  5.01 ms
654418347 amazon25bids30
'parse_auction_page'  0.07 ms
'parse_auction_table_page'  3.86 ms
'parse_history_page'  7.60 ms
722462041 shell50bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  6.85 ms
'parse_history_page'  29.09 ms
801043472 shell50bids35
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  6.18 ms
'parse_history_page'  30.35 ms
328662729 shell25bids20
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  5.79 ms
'parse_history_page'  15.79 ms
111405165 target25bids20
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  3.91 ms
'parse_history_page'  7.31 ms
899558584 target25bids30
'parse_auction_page'  0.05 ms
'parse_auction_table_page'  5.88 ms
'parse_history_page'  8.68 ms
972834507 walmart50bids30
'parse_auction_page'  0.06 ms
'parse_auction_table_page'  12.16 ms
'parse_history_page'  150.68

In [96]:
import pandas
bids_df = pd.DataFrame(list(bids_collection.find()))
auction_df = pd.DataFrame(list(auction_collection.find()))

In [97]:
auction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 10 columns):
_id                222 non-null int64
qauctionid         222 non-null object
cashvalue          222 non-null int64
cardvalue          222 non-null int64
bidvalue           222 non-null int64
cardtype           222 non-null object
limited_allowed    222 non-null bool
tracking           222 non-null float64
auctiontime        222 non-null datetime64[ns]
runtime            222 non-null int64
dtypes: bool(1), datetime64[ns](1), float64(1), int64(5), object(2)
memory usage: 16.0+ KB


In [98]:
auction_df["runtime"][0]

1619

In [99]:
for i in range(6):
    print(auction_df["auctiontime"][i])
    print(auction_df["runtime"][i])
   # bozo = datetime.strptime(auction_df["auctiontime"][i], '%m/%d/%Y %I:%M:%S %p')
    print(bozo)

2020-01-26 16:42:11
1619
2020-01-26 13:35:06
2020-01-26 15:38:23
5713
2020-01-26 13:35:06
2020-01-26 15:29:19
2047
2020-01-26 13:35:06
2020-01-26 12:34:28
2357
2020-01-26 13:35:06
2020-01-26 11:10:00
288
2020-01-26 13:35:06
2020-01-26 13:35:06
784
2020-01-26 13:35:06


In [43]:
auction_df["auctiontime"][auction_df["auctiontime"][2:]=="AM"][0]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [44]:
auction_df["auctiontime"][2:]

2       1/26/2020 3:29:19 PM
3      1/26/2020 12:34:28 PM
4      1/26/2020 11:10:00 AM
5       1/26/2020 1:35:06 PM
6      1/26/2020 11:44:12 AM
               ...          
217      2/1/2020 4:14:37 PM
218      2/1/2020 5:21:50 PM
219      2/1/2020 5:18:09 PM
220      2/1/2020 5:31:08 PM
221      2/1/2020 5:35:12 PM
Name: auctiontime, Length: 220, dtype: object

In [60]:
auction_df["runtime"]

0      00:26:59
1      01:35:13
2      00:34:07
3      00:39:17
4      00:04:48
         ...   
217    00:16:27
218    00:49:45
219    00:05:14
220    00:11:22
221    00:04:28
Name: runtime, Length: 222, dtype: object

In [215]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["penny"]
auction_collection = db["auctions"]
bids_collection = db["bids"]

conn = pg2.connect(user='postgres',  dbname='penny', host='localhost', port='5432', password='')
with conn.cursor() as cursor:
 
    sql = "SELECT qauctionid from auctions"
    cursor.execute(sql)
    already_found_qids = [x[0] for x in cursor.fetchall()]


    for d in auction_collection.find({ "_id": { "$nin":  already_found_qids}}):
        if d["tracking"] == 100.00:
            lock_price = d['lock_price']
            if (not lock_price):
                lock_price = 0
            sql = "INSERT INTO Auctions (qauctionid, cardvalue, bidvalue, tracking, auctiontime, runtime, limited_allowed, cashvalue, cardtype, lock_price) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (d['_id'], d['cardvalue'], d['bidvalue'], d['tracking'], d['auctiontime'], d['runtime'], d['limited_allowed'], d['cashvalue'], d['cardtype'], lock_price))
    conn.commit()

    sql = "SELECT distinct auctionid from bids"
    cursor.execute(sql)
    already_found_qids = [x[0] for x in cursor.fetchall()]
  
    for b in bids_collection.find({ "auction_id": { "$nin":  already_found_qids}}):
        sql = "INSERT INTO bids (auctionid, bid, username, is_bidomatic) VALUES (%s, %s, %s, %s)"
        cursor.execute(sql, (b['auction_id'], b['bid'], b['user'], b['is_bidomatic']))
        
    conn.commit()
    
    
conn.close()

In [165]:
for p in pages_collection.find({"_id": "218837987"}):
    print(p['AuctionTable'])

    

    <table style="width:100%;border-collapse:collapse;">
        <tr>
            <td> 100.00% Tracking </td>
            <td style="text-align:center"> Last Engine update: 1/26/2020 11:44:12 AM</td>
            <td style="text-align:center" id="idactive_bidders"> Active Bidders: 0</td>         
            <td style="text-align:right"> Auction runtime : 01:20:06</td>               
            <td id="refresh_time"></td>            
        </tr>
    </table>

    <div style="padding-bottom:5px;">
        
    <a href="History?site=quibids&auction_id=218837987"> Full History </a> &nbsp; &nbsp;<a href=AuctionListByDesc?site=quibids&hash=1119122541> Similar Items </a> &nbsp; &nbsp;<a href=QChooseit?auction_id=218837987> Choose It Options</a>    
   
    </div>
    <div id="div_progress" ></div> 
    
    
    <div id="div_auction_ended" style="display: none;">Ended</div>
     
   
    <div id="id_cdf" class="cdfbar" style="display:none;"" " >
        
        <div class="cdfbarlef

In [196]:
auction_collection[1]

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'penny'), 'auctions.1')